In [1]:
%load_ext autoreload
%autoreload 2

import sys

from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=['--ngrok_http_tunnel'] if 'google.colab' in sys.modules else [])

In [9]:
# This first cell has some updates/additions to the ConvexSet class hierarchy
# (which I aim to push in some form to drake).

import numpy as np
from pydrake.all import (
    ConvexSet, HPolyhedron, HyperEllipsoid,
    MathematicalProgram, Solve, le,
)

# Compute the "center" of a HPolyhedron.
def center(self):
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(self.ambient_dimension())
    r = prog.NewContinuousVariables(1)[0]

    A_row_norm = np.linalg.norm(self.A(), axis=1)
    lhs = self.A().dot(x) + A_row_norm * r
    prog.AddLinearConstraint(le(lhs, self.b()))
    prog.AddLinearConstraint(r >= 0)
    prog.AddLinearCost(-r)

    result = Solve(prog)
    assert result.is_success()

    return result.GetSolution(x)

HPolyhedron.center = center

def InscribedEllipse(self):
    ## Maximize inscribed ellipse
    prog = MathematicalProgram()
    A = self.A()
    b = self.b()
    C = prog.NewSymmetricContinuousVariables(3,'C')
    d = prog.NewContinuousVariables(3, 'd')
    prog.AddMaximizeLogDeterminantSymmetricMatrixCost(C)
    prog.AddPositiveSemidefiniteConstraint(C)
    for i in range(len(b)):
        prog.AddLorentzConeConstraint(np.concatenate((
            [b[i] - A[i,:].dot(d)],
            A[i,:] @ C
        )))
    result = Solve(prog)
    assert result.is_success()

    return HyperEllipsoid(np.linalg.inv(result.GetSolution(C)), result.GetSolution(d))

HPolyhedron.InscribedEllipse = InscribedEllipse

# TODO: Add a new constructor?  Or a static method?
def HPolyhedron_from_bounds(lb, ub):
    d = len(lb)
    assert len(ub) == d
    I = np.identity(d)
    return HPolyhedron(np.vstack((I, -I)), np.concatenate((ub, -lb)))

def GrowthVolume(self, set):
    # TODO: Robin super-optimized this step for v-rep obstacles (cvxgen, etc).
    assert self.ambient_dimension() == set.ambient_dimension()
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(self.ambient_dimension())
    set.AddPointInSetConstraint(prog, x)
    prog.AddQuadraticErrorCost(self.A().T @ self.A(), self.center(), x)
    result = Solve(prog)
    return result.get_optimal_cost(), result.GetSolution(x)

HyperEllipsoid.GrowthVolume = GrowthVolume

# Maybe this one doesn't actually deserve to be part of the main class,
# or it needs to be renamed.
# It's really the gradient of solution to the GrowthVolume optimization.
def TangentPlane(self, point):
    a = 2 * self.A().T @ self.A() @ (point - self.center())
    a = a / np.linalg.norm(a)
    b = a.dot(point)
    return a, b

HyperEllipsoid.TangentPlane = TangentPlane

import meshcat.geometry as g
from pydrake.all import RotationMatrix, RigidTransform
from scipy.spatial import ConvexHull, HalfspaceIntersection

def set_meshcat_object(meshcat, set, color=0xdd9999, wireframe=False):
    material = g.MeshLambertMaterial(color=color, wireframe=wireframe)

    if isinstance(set, np.ndarray):  # Render a point as a sphere.
        meshcat.set_transform(RigidTransform(set).GetAsMatrix4())
        meshcat.set_object(g.Sphere(.01), material)
    elif isinstance(set, HPolyhedron):
        assert set.ambient_dimension() == 3
        halfspaces = np.column_stack((set.A(), -set.b()))
        P = HalfspaceIntersection(halfspaces, set.center())
        vertices = P.intersections
        hull = ConvexHull(vertices)  # orders vertices counterclockwise
        vertices = vertices[hull.vertices]
        faces = hull.simplices
        meshcat.set_object(g.TriangularMeshGeometry(vertices, faces), material)
    elif isinstance(set, HyperEllipsoid):
        shape, X_WG = set.ToShapeWithPose()
        meshcat.set_transform(X_WG.GetAsMatrix4())
        meshcat.set_object(g.Ellipsoid([shape.a(), shape.b(), shape.c()]), material)
    else:
        print(type(set))
        raise NotImplementedError



In [10]:
# This cell builds a default environment for testing, with the planar kuka
# reaching into a shelf.  It's very nice, because it has a 3D configuration
# space, which I can visualize without projections.

from pydrake.all import (
    DiagramBuilder, AddMultibodyPlantSceneGraph, Parser,
    MathematicalProgram, Solve,
    ConnectMeshcatVisualizer,
    Role, Sphere
)
from manipulation.scenarios import AddShape, AddPlanarIiwa, AddWsg
from manipulation.utils import FindResource

def make_environment(robot=True, gripper=True, goal=True):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    if robot:
        iiwa = AddPlanarIiwa(plant, simple_collision=True)
        if gripper:
            wsg = AddWsg(plant, iiwa, roll=0.0, welded=True)
    if goal:
        sphere = AddShape(plant, Sphere(0.02), "sphere", collision=False)
        X_WO = RigidTransform([0.6, 0, 0.65])
        plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("sphere"), X_WO)

    parser = Parser(plant)
    shelves = parser.AddModelFromFile(FindResource("models/shelves.sdf"))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("shelves_body", shelves), RigidTransform([0.6,0,0.4]))

    plant.Finalize()

    visualizer = ConnectMeshcatVisualizer(
        builder,
        scene_graph,
        zmq_url=zmq_url,
        delete_prefix_on_load=False)

    diagram = builder.Build()
    visualizer.load()
    context = diagram.CreateDefaultContext()
    diagram.Publish(context)

    return diagram, plant, scene_graph

In [11]:
# This cell implements/tests IRIS in 3D (ignoring the robot geometry).
# This is a very small scale problem, but it works really well!

from meshcat import Visualizer
from pydrake.all import GeometrySet

def iris3d(query, point, lb, ub, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
    vis = Visualizer(zmq_url=zmq_url)
    set_meshcat_object(vis['sample'], point, color=0x99dd99)

    ellipsoid_epsilon = 1e-1
    dim = 3
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = HyperEllipsoid(np.eye(3)/ellipsoid_epsilon, point)
    set_meshcat_object(vis['ellipse'], E)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere

    inspector = query.inspector()
    num_geometries = inspector.NumGeometriesWithRole(Role.kProximity)

    P = HPolyhedron_from_bounds(lb, ub)
    A = np.vstack((np.zeros((num_geometries,3)),P.A()))
    b = np.concatenate((np.zeros(num_geometries),P.b()))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = [HPolyhedron.MakeFromSceneGraph(query, geom) for geom in geom_ids]

    iteration = 0
    while True:

        ## Find separating hyperplanes

        # Closest points
        closest = [E.GrowthVolume(set) for set in sets]
        closest.sort()

        # TODO: Remove redundant planes by checking if the obstacle is already outside the polytope in this loop.
        # Could use AABB to make it fast; the IRIS paper made a mistake, I think, by only checking the points.
        for i,c in enumerate(closest):
            distance, xstar = c
            set_meshcat_object(vis['closest'][f'{i}'], xstar)
            A[i,:], b[i] = E.TangentPlane(xstar)

        if any([np.any(A @ x > b) for x in require_containment_points]):
            print("terminating because a required containtment point would have not been contained")
            break

        P = HPolyhedron(A,b)
        set_meshcat_object(vis[f'polytope'], P, wireframe=True)

        ## Maximize inscribed ellipse

        E = P.InscribedEllipse()
        set_meshcat_object(vis[f'ellipse'], E)
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P

np.set_printoptions(formatter={'float': lambda x: "{0:0.1f}".format(x)})

diagram, plant, scene_graph = make_environment(robot=False)
lb = np.array([0, -.5, 0])
ub = np.array([1, .5, 1])
x = np.array([0.5, 0, 0.45])
#x = np.array([0.3, 0, 0.45])
context = diagram.CreateDefaultContext()
query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))
iris3d(query, x, lb, ub, require_containment_points=[x], iteration_limit=100);

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
0
1


In [12]:
# Now IRIS in configuration space, using dReal to solve for the growth volume
# through the nonconvex kinematics.

from pydrake.all import (
    DrealSolver, eq, SnoptSolver,
    Sphere, Ellipsoid,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)
from IPython.display import display

diagram, plant, scene_graph = make_environment(robot=True, gripper=False)
lb = np.array([0, -.5, 0])
ub = np.array([1, .5, 1])
context = diagram.CreateDefaultContext()
query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))

sym_plant = plant.ToSymbolic()
sym_context = sym_plant.CreateDefaultContext()
dReal = DrealSolver()

# For SNOPT test.
autodiff_plant = plant.ToAutoDiffXd()
autodiff_context = autodiff_plant.CreateDefaultContext()
snopt = SnoptSolver()

vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())

def MakeFromSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return HyperEllipsoid.MakeFromSceneGraph(query, geom, expressed_in)
    return HPolyhedron.MakeFromSceneGraph(query, geom, expressed_in)


def GrowthVolume(E, bodyA, bodyB, setA, setB, A, b):
    prog = MathematicalProgram()
    q = prog.NewContinuousVariables(plant.num_positions(), "q")
    prog.AddBoundingBoxConstraint(lb, ub, q)
    prog.AddLinearConstraint(A, b-np.inf, b, q)
    p_AA =  prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), q)
    setA.AddPointInSetConstraint(prog, p_AA)
    setB.AddPointInSetConstraint(prog, p_BB)

    # TODO: Remove these.  They're for debugging only.
    set_meshcat_object(vis['setA'],setA)
    set_meshcat_object(vis['setB'],setB)

    if isinstance(bodyA, RigidBody_[AutoDiffXd]):
        print("using autodiff")
        def kinematics_constraint(vars):
            p_AA, p_BB, q = np.split(vars,[3,6])
            autodiff_plant.SetPositions(autodiff_context, q)
            X_WA = autodiff_plant.EvalBodyPoseInWorld(autodiff_context, bodyA)
            X_WB = autodiff_plant.EvalBodyPoseInWorld(autodiff_context, bodyB)
            p_WA = X_WA.multiply(p_AA)
            p_WB = X_WB.multiply(p_BB)
            return p_WA - p_WB
        prog.AddConstraint(kinematics_constraint, lb=[0,0,0], ub=[0,0,0],
                           vars=np.concatenate((p_AA,p_BB,q)))
        result = snopt.Solve(prog)

    else:
        # TODO: Construct these symbolic expressions once per body outside this method.
        # But I would have to substitute in the new q each time.
        # Better is to construct the prog once for each pair, and just update the cost?
        sym_plant.SetPositions(sym_context, q)
        X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, bodyA)
        X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, bodyB)
        # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
        p_WA = X_WA.multiply(p_AA+0)
        p_WB = X_WB.multiply(p_BB+0)
        prog.AddConstraint(eq(p_WA, p_WB))
        result = snopt.Solve(prog)
        #result = dReal.Solve(prog)

    return result.is_success(), result.get_optimal_cost(), result.GetSolution(q)

def iris_cspace(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
    vis = Visualizer(zmq_url=zmq_url)
    set_meshcat_object(vis['sample'], point, color=0x99dd99)

    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    ub = plant.GetPositionUpperLimits()
    assert dim == 3 # need to update the volume once this changes
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = HyperEllipsoid(np.eye(3)/ellipsoid_epsilon, point)
    set_meshcat_object(vis['ellipse'], E)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron_from_bounds(lb, ub)
    A = np.vstack((P.A(), np.zeros((len(pairs),3))))
    b = np.concatenate((P.b(), np.zeros(len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}

    use_autodiff = False
    if use_autodiff:
        bodies = {geom:autodiff_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
    else:
        bodies = {geom:sym_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}

    iteration = 0
    while True:
        ## Find separating hyperplanes

        num_faces = 2*len(lb)
        for geomA, geomB in pairs:
            reachable, growth, qstar = GrowthVolume(E,
                bodies[geomA], bodies[geomB],
                sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces])
            if reachable:
                A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                num_faces += 1

        if any([np.any(A[:num_faces,:] @ q > b[:num_faces]) for q in require_containment_points]):
            print("terminating because a required containtment point would have not been contained")
            break

        print(A[:num_faces,:])
        print(b[:num_faces])
        P = HPolyhedron(A[:num_faces,:],b[:num_faces])
        set_meshcat_object(vis[f'polytope'], P, wireframe=True)

        E = P.InscribedEllipse()
        set_meshcat_object(vis[f'ellipse'], E)
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P


q = plant.GetPositions(plant.GetMyContextFromRoot(context))
iris_cspace(query, q, require_containment_points=[q], iteration_limit=100);


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
[[1.0 0.0 0.0]
 [0.0 1.0 0.0]
 [0.0 0.0 1.0]
 [-1.0 -0.0 -0.0]
 [-0.0 -1.0 -0.0]
 [-0.0 -0.0 -1.0]
 [0.5 0.7 -0.6]]
[2.1 2.1 2.1 2.1 2.1 2.1 -0.7]
0
[[1.0 0.0 0.0]
 [0.0 1.0 0.0]
 [0.0 0.0 1.0]
 [-1.0 -0.0 -0.0]
 [-0.0 -1.0 -0.0]
 [-0.0 -0.0 -1.0]
 [0.6 0.7 -0.4]]
[2.1 2.1 2.1 2.1 2.1 2.1 -0.4]
1
[[1.0 0.0 0.0]
 [0.0 1.0 0.0]
 [0.0 0.0 1.0]
 [-1.0 -0.0 -0.0]
 [-0.0 -1.0 -0.0]
 [-0.0 -0.0 -1.0]
 [0.7 0.7 -0.1]]
[2.1 2.1 2.1 2.1 2.1 2.1 -0.1]
2
[[1.0 0.0 0.0]
 [0.0 1.0 0.0]
 [0.0 0.0 1.0]
 [-1.0 -0.0 -0.0]
 [-0.0 -1.0 -0.0]
 [-0.0 -0.0 -1.0]
 [0.8 0.6 0.1]]
[2.1 2.1 2.1 2.1 2.1 2.1 0.3]
3
[[1.0 0.0 0.0]
 [0.0 1.0 0.0]
 [0.0 0.0 1.0]
 [-1.0 -0.0 -0.0]
 [-0.0 -1.0 -0.0]
 [-0.0 -0.0 -1.0]
 [1.0 0.3 0.1]]
[2.1 2.1 2.1 2.1 2.1 2.1 0.5]
4
[[1.0 0.0 0.0]
 [0.0 1.0 0.0]
 [0.0 0.0 1.0]
 [-1.0 -0.0 -0.0]
 [-0.0 -1.0 -0.0]
 [-0.0 -0.0 -1.0]
 [1.0 0.0 0.1]]
[2.1 2.1 2.1 2.1 2.1 2.1 0.7]
5
[[1.0 0.0 0.0]
 [0.0 1.

In [7]:
from meshcat import Visualizer
from pydrake.all import (
    DiagramBuilder, AddMultibodyPlantSceneGraph, SpatialInertia, RigidTransform,
    CoulombFriction, Role,
)
from IPython.display import display

def MakeSceneGraphWithShape(shape, X_WG):
  builder = DiagramBuilder()
  plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)
  instance = plant.AddModelInstance("test")
  body = plant.AddRigidBody("test", instance, SpatialInertia())
  geom_id = plant.RegisterCollisionGeometry(
      body, RigidTransform.Identity(), shape, "test",
      CoulombFriction())
  plant.WeldFrames(plant.world_frame(), body.body_frame(), X_WG)
  plant.Finalize()
  return builder.Build(), scene_graph, geom_id

def TestInscribedEllipse():
    H = HPolyhedron_from_bounds(np.array([-1,-1,-1]), np.array([1,1,1]))
    E = H.InscribedEllipse()
    assert np.allclose(E.A(), np.eye(3))
    assert np.allclose(E.center(), [0,0,0])

    A = np.vstack((np.eye(3), -np.eye(3), [.9, -.3, .1], [.9, -.3, .1]))
    b = np.array([2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 1.3, 0.8])
    H = HPolyhedron(A, b)
    E = H.InscribedEllipse()
    out1 = np.array([1.4, 0, 0])
    out2 = np.array([-5, 0, 0])
    assert not H.PointInSet(out1)
    assert not H.PointInSet(out2)
    assert not E.PointInSet(out1)
    assert not E.PointInSet(out2)

    shape, X_WS = E.ToShapeWithPose()
    diagram, scene_graph, geom_id = MakeSceneGraphWithShape(shape, X_WS)
    context = diagram.CreateDefaultContext()
    query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))
    E2 = HyperEllipsoid.MakeFromSceneGraph(query, geom_id)
    shape2, X_WS2 = E2.ToShapeWithPose()

    # Known issues/questions:
    # - the query.ComputeSignedDistanceToPoint call below doesn't return any values...

    print(query.inspector().NumGeometriesWithRole(Role.kProximity))
    print(query.ComputeSignedDistanceToPoint([10, 10, 10]))
    #assert query.ComputeSignedDistanceToPoint(out1)[0] > 0.0
    #assert query.ComputeSignedDistanceToPoint(out2)[0] > 0.0

    vis = Visualizer(zmq_url=zmq_url)
    vis.delete()
    set_meshcat_object(vis['polytope'], H, wireframe=True)
    set_meshcat_object(vis['ellipse'], E)
    set_meshcat_object(vis['ellipse2'], E2)
    #display(vis.jupyter_cell())

TestInscribedEllipse()

[[ 0.68657286 -0.06762867  0.02230509]
 [-0.06762867  0.48523115 -0.00759985]
 [ 0.02230509 -0.00759985  0.47724077]]
[-6.05546104e-01  1.23571389e-08  3.61827368e-09]
RigidTransform(
  R=RotationMatrix([
    [0.28382015036009384, 0.11784216564108617, 0.9516088199709966],
    [0.9560345302225315, 0.041588217399216594, -0.2902901947977908],
    [-0.07378413970331009, 0.9921610978921592, -0.10085760535362756],
  ]),
  p=[-0.6055461044095725, 1.2357138867713502e-08, 3.618273680672246e-09],
)
[2.15253955675936, 2.1056617455866165, 1.4093098584396213]
[[ 0.13185363  0.44414261 -0.03427772]
 [ 0.05596443  0.01975066  0.47118731]
 [ 0.67523037 -0.20598039 -0.07156525]]
[-6.05546104e-01  1.23571388e-08  3.61827360e-09]
RigidTransform(
  R=RotationMatrix([
    [0.283820150360093, -0.11784216564108806, 0.9516088199709966],
    [0.9560345302225315, -0.041588217399223214, -0.2902901947977908],
    [0.07378413970331703, 0.9921610978921588, 0.10085760535362752],
  ]),
  p=[-0.6055461044095725, 1.235